In [1]:
!pip install bs4

In [2]:
!pip install fake_useragent

In [ ]:
# Basic libraries
import pandas as pd
import numpy as np
import nltk

# Importing for scraping
from bs4 import BeautifulSoup 
import fake_useragent


import time         # Tracking time
import requests     # HTTP requests
import re           # String manipulation

In [ ]:
def scrape_page(page_url):
    """Extracts HTML from a webpage"""
    user_agent = fake_useragent.UserAgent().random
    headers = dict(user_agent=user_agent)
    answer = requests.get(page_url,headers= headers)
    html = answer.content
    return html

In [ ]:
def html2text(html):
    soup = BeautifulSoup(html, features='html.parser')
    text = soup.get_text()
    return text.split('SENTENZA')[-1].replace('\n',' ')

In [ ]:
html2text(scrape_page('https://www.giustizia-amministrativa.it/portale/pages/istituzionale/visualizza/?nodeRef=&schema=tar_rm&nrg=202002103&nomeFile=202014201_01.html&subDir=Provvedimenti'))

' sul ricorso numero di registro generale 2103 del 2020, proposto da ALESSANDRO SCROCCA elettivamente domiciliato in Roma, largo Messico n. 7 presso lo studio dell’avv. Alessandro Tozzi che lo rappresenta e difende nel presente giudizio  contro ROMA CAPITALE, in persona del Sindaco p.t., elettivamente domiciliata in Roma, via del Tempio di Giove n. 21 presso la sede dell’avvocatura comunale e rappresentata e difesa nel presente giudizio dall’avv. Michele Memeo   nei confronti - ALESSANDRO MOLINARI elettivamente domiciliato in Roma, via A. Gramsci n. 7 presso lo studio dell’avv. Giorgio Leccisi che lo rappresenta e difende nel presente giudizio;- SOGGETTI CHE HANNO CONSEGUITO UNA POSIZIONE IN GRADUATORIA MIGLIORE DI QUELLA DEL RICORRENTE, già destinatari dell’ordinanza n. 3383/2020 con cui è stata disposta l’integrazione del contraddittorio nei loro confronti – non costituiti in giudizio;  per l\'annullamento dei seguenti atti: - determinazione dirigenziale n. QD 3137 del 18/12/2019 di 

## Crawling  following [link](https://www.giustizia-amministrativa.it/web/guest/dcsnprr)

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
from selenium.webdriver.support.ui import Select

In [ ]:
import random

In [ ]:
def search(url_provvedimenti):
    driver.get(url_provvedimenti)

    select = Select(driver.find_element_by_id('_GaSearch_INSTANCE_2NDgCF3zWBwk_pageResultsProvvedimenti'))

    # select by visible text
    select.select_by_visible_text('60')

    select = Select(driver.find_element_by_id('_GaSearch_INSTANCE_2NDgCF3zWBwk_TipoProvvedimentoItem'))

    # select by visible text
    select.select_by_visible_text('Sentenza')

    driver.find_element_by_id("_GaSearch_INSTANCE_2NDgCF3zWBwk_submitButton").click()

    return driver

In [ ]:
d = search('https://www.giustizia-amministrativa.it/web/guest/dcsnprr')
d.implicitly_wait(random.randint(2,5))
links =  [x.find_element_by_css_selector('a').get_attribute('href') for x in driver.find_elements_by_class_name("ricerca--item")][:-1]
for i in range(30):
    d.find_elements_by_class_name("pagination-li ")[2].click()
    links.extend([x.find_element_by_css_selector('a').get_attribute('href') for x in d.find_elements_by_class_name("ricerca--item")][:-1])
    


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_ele

In [ ]:
len(links)

960

In [ ]:
htmls = []
texts = []

for link in links:
    htmls.append(scrape_page(link))
    texts.append(html2text(htmls[-1]))

df_sentenze = pd.DataFrame(
                            {'link': links,
                            'html': htmls,
                            'text': texts
                            }
                           )


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_sentenze.nunique()

link    960
html    949
text    948
dtype: int64

In [ ]:
df_sentenze.drop_duplicates(subset='text',inplace= True)

In [ ]:
df_sentenze.nunique()

link    948
html    948
text    948
dtype: int64

In [ ]:
df_sentenze.to_csv('/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization/data/sentenze_html.csv',index = False)